In [1]:
import pandas as pd
import numpy as np
from pandas import datetime

#### Import data and convert date format

In [2]:
# date-time parsing function for loading the dataset
def parserSTOCK(x):
	return datetime.strptime(x, '%Y%m%d')
def parserFF(x):
	return datetime.strptime(x, '%Y%m')
stock = pd.read_csv('cleaned_data.csv',index_col=0, parse_dates=[4], date_parser=parserSTOCK).rename(columns={'time_stamp':'date'})
FF = pd.read_csv('Fama French 5 Factors.CSV').rename(columns={'Unnamed: 0':'date'})
FF=FF[FF['date'].astype(str).str.len()==6]
FF['date']=FF['date'].astype(str)
FF['date']=FF['date'].apply(lambda x:datetime.strptime(x, '%Y%m'))
PS = pd.read_csv('Pastor Stambaugh Factors.csv', parse_dates=[0], date_parser=parserSTOCK).rename(columns={'DATE':'date'})
HXZ = pd.read_excel('HXZ q-Factors (monthly 1967 to 2014).xlsx')
HXZ['date'] = (HXZ['Year']*100+HXZ['Month']).astype(str)
HXZ['date']=HXZ['date'].apply(lambda x:datetime.strptime(x, '%Y%m'))
HXZ=HXZ.drop(columns=['Year','Month'])
# pd.to_datetime(dict(year=df.Y, month=df.M, day=df.D))

#### Filter data to only keep Jan1992 - Dec2012

In [3]:
def Filter(df):
    return df[(df['date'].dt.year>=1992)&(df['date'].dt.year<=2012)].reset_index(drop=True)
stock=Filter(stock)
FF=Filter(FF)
PS=Filter(PS)
HXZ=Filter(HXZ)
PS.iloc[:,1:] = PS.iloc[:,1:]*100

#### Evaluate the correlation of different factors

In [4]:
def Calibrate_time(df):
    return df['date']+pd.offsets.MonthEnd(0) 
FF['date']=Calibrate_time(FF) 
PS['date']=Calibrate_time(PS) 
HXZ['date']=Calibrate_time(HXZ)
All_factors = pd.merge(FF,PS,how='inner',on='date')
All_factors = pd.merge(All_factors,HXZ,how='inner',on='date').set_index('date').astype(float)

In [123]:
from pandas_profiling import ProfileReport

profile = ProfileReport(All_factors, title='Factor Description', html={'style':{'full_width':True}})
profile.to_file(output_file="Factor Descrption.html")
profile=profile.to_notebook_iframe()

In [7]:
stock.to_csv('stock.csv')